# Product History Table

- p_id from **1 to 100** already present in  main table, it can be updated and deleted
- p_id from **101 to 200** can be inserted in to main table and after inserting it into main table it could also be deleted


### Libraries Used

In [1]:
import datetime
import os

# Main entry point for DataFrame and SQL functionality.
from pyspark.sql import SparkSession
# Start SPARK Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.types import *


### Read the Main Table

In [2]:
# Read from main_table
Table = spark.read.format('csv').options(
    header=True, inferschema=True).load(
        "/home/bluepi/Downloads/Update/product_info/main table/main_table_new.csv")

# Add record type to main table
Table_new = Table.withColumn('record_type', lit("A"))

# Write in main_table folder
path = "/home/bluepi/Downloads/Update/Updated Product/Product History/"
# try:
#     Table_new.write.format('csv').save(
#         os.path.join(path, 'past_days'), header=True)
# except:
#     Table_new.write.mode('overwrite').format('csv').save(
#         os.path.join(path, 'past_days'), header=True)


### Main Code

In [3]:
# Storing all the p_id in a list which we have deleted and inserted
total_products_inserted_list = []
total_products_deleted_list = []

data_schema = [
    StructField('p_id', IntegerType(), True),
    StructField('p_name', StringType(), True),
    StructField('price', IntegerType(), True),
    StructField('date_timestamp', TimestampType(), True),
    StructField('record_type', StringType(), True)
    ]
# Schema which we are accepting
final_struc = StructType(fields=data_schema)
# day_count = 10

for i in range(10, -1, -1):

    # In this try-catch block we are only taking the previous day data, but initially there is
    # no previous day, therefore we are taking main table at begining
    try:
        path1 = os.path.join(
            "/home/bluepi/Downloads/Update/product_info/", t)
        mainTable = spark.read.csv(path1, header=True, schema=final_struc)
        print("The per day data path is \n"+path1, end='\n')

    except:
        Table_new.write.format('csv').save(
        os.path.join(path, 'past_days'), header=True)
        print("The past days data path is \n"+path, end='\n')
#     print("\n File read from --")
#     print(path1, end='\n')

    # Address to the product_info folder
    address = "/home/bluepi/Downloads/Update/product_info/"
    previous_day = datetime.datetime.today() - datetime.timedelta(days=i)
    #print("\nDate ----> "+previous_day+"\n")

    # before_insert = str(mainTable.count())
    #print("Main_Table Count ----> " + str(mainTable.count()))

    # Address to the Previous Day folder
    new_address = address + previous_day.strftime('%d-%m-%Y')
#     print("\nNew Address to read the folder ---->"+new_address)



# ###################################################################################################
    
    try:
        previous_day_new = (previous_day - datetime.timedelta(days=1)).strftime('%d-%m-%Y')
        
        # If folder already exist, then no need to move forward
        if os.path.isdir(os.path.join(path, previous_day_new)) :
            print("{} date folder already".format(previous_day_new))
        
        # If folder is not present
        else :
            
#             # We do not want date to be printed when past_days folder created
#             if i != 10:
#                 print("\nDate ----> "+previous_day_new +"\n")
                
    #         print("DAY COUNT IS {}\n".format(day_count))
            print("\nTotal products in {} Date Table are {}".format(previous_day_new,mainTable.count()))
            print("\nRetrive Inserted Products")
            
        # Get the list of p_id which we have to inserted in the main table
            mainTable_I_New = mainTable.filter("record_type == 'I' ")
            p_id_list_I = [i.p_id for i in mainTable_I_New.collect()]
            print("Total no. of products \"Inserted\" :- {}".format(len(p_id_list_I)))
    
        # We can remove this section if not required
        # Since we don't want our Inserted product to be Inserted again
            for i in total_products_inserted_list:
                if i in p_id_list_I:
                    print("\n{} is going to be inserted again\n".format(i))
                # Therefore removing already inserted products
                    p_id_list_I.remove(i)
                    mainTable_I_New = mainTable_I_New.filter(mainTable.p_id != i)
                    
                    
            print("Total no. of products \"Inserted\"(removing already inserted) :- {}".format(len(p_id_list_I)))
        
        # Storing all the p_id in a list which we have inserted
            total_products_inserted_list.extend(p_id_list_I)
    
    #         mainTable_I = mainTable.filter(mainTable.p_id.isin(list(set(p_id_list_I))))
            print("Total products in main_Table_I are {}".format(mainTable_I_New.count()))
            print("Retrived Inserted Products\n")
    
# ###################################################################################################
    
            print("Retrive Updated Products")
            #print("\nTable of Products to be Updated in Main_Table taken from Per_Day_Table \n")
    #           per_day_data.filter("record_type == 'U' ").orderBy(per_day_data.p_id.asc()).show()
    
            # Get the list of p_id which we have to updated in the main table
            to_be_updated = mainTable.filter("record_type == 'U' ").collect()
            p_id_list_U = [i.p_id for i in to_be_updated]
            print("Total no. of products before\"Updating\" :- {}".format(len(p_id_list_U)))
    
            # Created a new DataFrame of records to be updated
            from_per_day_data_U = mainTable.filter("record_type == 'U' ")
    
            for_join_mT_and_pDD = from_per_day_data_U.groupBy("p_id").agg(
                    {"date_timestamp": "max"}).withColumnRenamed("max(date_timestamp)", "date_timestamp_1")
            for_join_mT_and_pDD = for_join_mT_and_pDD.withColumnRenamed("p_id", "p_id_1")
            # for_join_mT_and_pDD.orderBy(col("p_id_1")).show()
            
            # Performed Join opteration to pick only latest updates only
            joined = from_per_day_data_U.join(for_join_mT_and_pDD, (
                    from_per_day_data_U.p_id == for_join_mT_and_pDD.p_id_1) & (
                    from_per_day_data_U.date_timestamp == for_join_mT_and_pDD.date_timestamp_1), 'inner')
    
            mainTable_U = joined.select(
                    ['p_id', 'p_name', 'price', 'date_timestamp', 'record_type'])
            
            to_be_updated_U = mainTable_U.filter("record_type == 'U' ").collect()
            p_id_list_U_U = [i.p_id for i in to_be_updated_U]
            print("Total no. of products after\"Updating\" (removing multiples) :- {}".format(len(p_id_list_U_U)))
            
            # joined.orderBy(col("p_id")).show()
            print("Total products in main_Table_U are {}".format(mainTable_U.count()))
            print("Retrived Updated Products\n")
    
# ###################################################################################################
    
            print("Retrive Deleted Products")
        
        # Get the list of p_id which we have to delete
            mainTable_D_New = mainTable.filter("record_type == 'D' ")
            p_id_list_D = [i.p_id for i in mainTable_D_New.collect()]
            print("Total no. of products \"Deleted\" :- {}".format(len(p_id_list_D)))
            
        # We can remove this section if not required
        # Since we don't want our deleted product to be deleted again
            for i in total_products_deleted_list:
                if i in p_id_list_D:
                    print("\n{} is going to be deleted again\n".format(i))
                    # Therefore removing already deleted products
                    p_id_list_D.remove(i)
                    mainTable_D_New = mainTable_D_New.filter(mainTable.p_id != i)
               
            
            print("Total no. of products \"Deleted\"(removing already deleted) :- {}".format(len(p_id_list_D)))
            
            # Storing all the p_id in a list which we have deleted
            total_products_deleted_list.extend(p_id_list_D)
    
            # Remove the deleted p_id from main_table
    #         mainTable_D = mainTable.filter(col('p_id').isin(list(set(p_id_list_D))))
            print("Total products in main_Table_D are {}".format(mainTable_D_New.count()))
            print("Retrived Deleted Products\n")
            
# ###################################################################################################
    
            mainTable_New = mainTable_U.union(mainTable_I_New)
            mainTable_New = mainTable_New.union(mainTable_D_New)
            # mainTable_New.show()
    
                        
            print("{} contains {} products".format(previous_day_new,mainTable_New.count()))
            mainTable_New.coalesce(1).write.format('csv').save(
                os.path.join(path, t), header=True)
    #         day_count -= 1
    #         print("\nDate ----> "+previous_day_new+"\n")

    except:
        print("Past days folder contains {} products".format(Table_new.count()))

# ###################################################################################################

    t = str(previous_day.strftime('%d-%m-%Y'))
#     print("\n\t t is {} \n".format(t))

    print("\n***************************************************************************************\n")

The past days data path is 
/home/bluepi/Downloads/Update/Updated Product/Product History/
Past days folder contains 100 products

***************************************************************************************

The per day data path is 
/home/bluepi/Downloads/Update/product_info/04-04-2020

Total products in 04-04-2020 Date Table are 35

Retrive Inserted Products
Total no. of products "Inserted" :- 6
Total no. of products "Inserted"(removing already inserted) :- 6
Total products in main_Table_I are 6
Retrived Inserted Products

Retrive Updated Products
Total no. of products before"Updating" :- 25
Total no. of products after"Updating" (removing multiples) :- 22
Total products in main_Table_U are 22
Retrived Updated Products

Retrive Deleted Products
Total no. of products "Deleted" :- 4
Total no. of products "Deleted"(removing already deleted) :- 4
Total products in main_Table_D are 4
Retrived Deleted Products

04-04-2020 contains 32 products

***********************************

Total no. of products after"Updating" (removing multiples) :- 8
Total products in main_Table_U are 8
Retrived Updated Products

Retrive Deleted Products
Total no. of products "Deleted" :- 6

30 is going to be deleted again

Total no. of products "Deleted"(removing already deleted) :- 5
Total products in main_Table_D are 5
Retrived Deleted Products

13-04-2020 contains 25 products

***************************************************************************************



In [4]:
os.path.isdir(os.path.join(path, previous_day_new))

True